# 1. Compute Results Overview Tables

The success of an assembly is evaluated by the computation of metrics in two defined ways: globally through statistics inherent to the complete set of sequences that were assembled, and relative to the replicons present in the sample. 

The computation of the global metrics is done through custom python code from the complete set of sequences assembled. 

The following metrics are computed for the complete and filtered set of assembled sequences, restricted to contigs of length above a specified minimum size: 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Maximum sequence length:** The length of the largest contig in the assembly.
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **Contiguity**
    - **Nx (where 0  < x  ⩽ 100):** Length for which the collection of all assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the assembly
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event

After filtering, the sequences are mapped with the reference replicons and the metrics are computed through custom python code. The metrics are calculated, filtered for a minimum length, for each replicon in the file provided input references. 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **COMPASS**
    - **(Breadth of) Coverage:** Ratio of covered sequence on the reference by aligned contigs;
    - **Multiplicity:** Ratio of the length of alignable assembled sequence to covered sequence on the reference;
    - **Validity:** Ratio of the length of the alignable assembled sequence to total basepairs in the aligned contigs;
    - **Parsimony:** Cost of the assembly (multiplicity over validity);
- **Contiguity**
    - **Contiguity:** longest single alignment between the assembly and the reference, relative to the reference length;
    - **NAx (where 0  < x  ⩽ 100):** Length for which the collection of aligned assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the reference replicon;
    - **NGx (where 0  < x  ⩽ 100):** Length for which the collection of aligned contigs of that length or longer covers at least a given percentage of the sequence of the reference.
    - **Lx (where 0  < x  ⩽ 100):** Minimal number of contigs that cover x % of the sequence of the reference;
- **Identity**
    - **Identity:** Ratio of identical basepairs in all aligned contigs to the reference;
    - **Lowest identity:** Identity of the lowest scoring contig to the reference.
    - **PLS**: Phred-like score per contig, per assembler.
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event


## Imports

In [1]:
import sys
from plotly.offline import plot
import glob
import fnmatch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import pandas as pd
from itertools import groupby
import csv
import numpy as np

## Global variables

In [42]:
ASSEMBLER_PROCESS_LIST = ["ABYSS", "BCALM2", "GATBMINIAPIPELINE", "METAHIPMER2", "MINIA", "MEGAHIT", "METASPADES", "UNICYCLER", "SPADES",
                          "SKESA", "VELVETOPTIMIZER", "IDBA"]
PROCESS_TO_NAME = {"ABYSS": "AbYSS",
                   "BCALM2": "BCALM2", 
                   "GATBMINIAPIPELINE": "GATBMiniaPipeline",
                   "METAHIPMER2": "MetaHipMer2",
                   "MINIA": "MINIA",
                   "MEGAHIT": "MEGAHIT", 
                   "METASPADES": "metaSPAdes", 
                   "UNICYCLER": "Unicycler", 
                   "SPADES": "SPAdes",
                    "SKESA": "SKESA",
                   "VELVETOPTIMIZER": "VelvetOptimizer",
                   "IDBA": "IDBA-UD"}

log_distributed = ['LHS', 'LNN', 'ERR2935805']
even_distribution = ['EMS', 'LNN', 'ERR2984773']

genomic_assemblers = ['ABySS', 'BCALM2', 'MINIA', 'SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer']
metagenomic_assemblers = ['MetaHipMer2','GATBMiniaPipeline', 'IDBA-UD', 'MEGAHIT', 'metaSPAdes']
single_kmer = ['BCALM2', 'MINIA', 'ABySS']
multiple_kmer = ['SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer', 'GATBMiniaPipeline', 
                 'IDBA-UD', 'MEGAHIT', 'metaSPAdes', 'MetaHipMer2']

best_min = ['Ns', 'contigs', 'filtered_Ns', 'filtered_contigs','misassembled contigs','misassembly events']
best_max = ['basepairs','filtered_basepairs','filtered_mapped_reads','filtered_n50','mapped_reads','max_contig','n50']

## Global metrics

### Load data

In [29]:
report_glob = glob.glob('../Results/*/*/report/pipeline_report_tables.json')
global_pipeline_metrics_df = pd.DataFrame()

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for line in json_report[sample]['GlobalTable']:
                assembler = line['assembler']
                global_pipeline_metrics_df = global_pipeline_metrics_df.append({'run': stats_run,
                                                                                'sample': sample,
                                                                                'assembler': line['assembler'],
                                                                                'contigs': int(line['original']['contigs']),
                                                                                'basepairs': int(line['original']['basepairs']),
                                                                                'max_contig': int(line['original']['max_contig_size']),
                                                                                'n50': int(line['original']['N50']),
                                                                                'mapped_reads': line['original']['mapped_reads'],
                                                                                'Ns': int(line['original']['Ns']),
                                                                                'misassembled contigs': line['filtered']['misassembled_contigs'],
                                                                                'misassembly events': line['filtered']['misassembly_events'],
                                                                                'filtered_contigs': line['filtered']['contigs'],
                                                                                'filtered_basepairs': line['filtered']['basepairs'],
                                                                                'filtered_n50': line['filtered']['N50'],
                                                                                'filtered_Ns': line['filtered']['Ns'],
                                                                                'filtered_mapped_reads': line['filtered']['mapped_reads'],
                                                                               },
                                                                               ignore_index=True)
global_pipeline_metrics_df['distribution'] = np.where(global_pipeline_metrics_df['sample'].isin(log_distributed), 'Log', 'Even')
global_pipeline_metrics_df['type'] = np.where(global_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
global_pipeline_metrics_df['algorythm'] = np.where(global_pipeline_metrics_df['assembler'].isin(single_kmer), 'Single k-mer De Bruijn graph', 'Multiple k-mer De Bruijn graph')
global_pipeline_metrics_df[['contigs','basepairs','max_contig', 'Ns','n50', 'filtered_n50','misassembled contigs', 'misassembly events']] = global_pipeline_metrics_df[['contigs','basepairs','max_contig', 'Ns','n50','filtered_n50','misassembled contigs', 'misassembly events']].apply(pd.to_numeric)

Processing pipeline_report_tables.json data from run3...
Processing pipeline_report_tables.json data from run2...
Processing pipeline_report_tables.json data from run1...
Processing pipeline_report_tables.json data from run3...
Processing pipeline_report_tables.json data from run2...


### Preview data

In [30]:
global_pipeline_metrics_df.dtypes

Ns                       float64
assembler                 object
basepairs                float64
contigs                  float64
filtered_Ns              float64
filtered_basepairs       float64
filtered_contigs         float64
filtered_mapped_reads    float64
filtered_n50             float64
mapped_reads             float64
max_contig               float64
misassembled contigs     float64
misassembly events       float64
n50                      float64
run                       object
sample                    object
distribution              object
type                      object
algorythm                 object
dtype: object

In [31]:
global_pipeline_metrics_df

,Ns,assembler,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50,run,sample,distribution,type,algorythm
0,1345.0,ABySS,20132244.0,44619.0,400.0,13573790.0,1211.0,98.901529,192306.0,99.851184,851314.0,0.0,0.0,9569.0,run3,LNN,Log,Genomic,Single k-mer De Bruijn graph
1,0.0,BCALM2,19635512.0,44825.0,0.0,13397815.0,784.0,97.222124,31436.0,98.306950,163543.0,0.0,0.0,18682.0,run3,LNN,Log,Genomic,Single k-mer De Bruijn graph
2,0.0,GATBMiniaPipeline,13911245.0,693.0,0.0,13658336.0,219.0,98.807198,140561.0,99.719684,632215.0,1.0,3.0,136741.0,run3,LNN,Log,Metagenomic,Multiple k-mer De Bruijn graph
3,0.0,IDBA-UD,14324920.0,1987.0,0.0,13642314.0,261.0,98.951939,108043.0,99.694880,585610.0,2.0,2.0,105808.0,run3,LNN,Log,Metagenomic,Multiple k-mer De Bruijn graph
4,0.0,MEGAHIT,14402555.0,1842.0,0.0,13704539.0,172.0,99.686862,210993.0,99.765145,1080013.0,1.0,2.0,203548.0,run3,LNN,Log,Metagenomic,Multiple k-mer De Bruijn graph
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,0.0,MINIA,30648944.0,16748.0,0.0,28218335.0,3574.0,92.157082,20782.0,95.935753,161812.0,0.0,0.0,17967.0,run2,EMS,Even,Genomic,Single k-mer De Bruijn graph
212,0.0,SKESA,30249132.0,1269.0,0.0,30064811.0,891.0,97.237351,74138.0,97.952533,632013.0,9.0,18.0,73839.0,run2,EMS,Even,Genomic,Multiple k-mer De Bruijn graph
213,0.0,SPAdes,30664883.0,2628.0,0.0,30354786.0,412.0,98.283354,208991.0,98.700052,850176.0,0.0,0.0,206515.0,run2,EMS,Even,Genomic,Multiple k-mer De Bruijn graph
214,0.0,Unicycler,30301867.0,656.0,0.0,30188943.0,334.0,97.609864,245050.0,98.457135,1303875.0,0.0,0.0,245050.0,run2,EMS,Even,Genomic,Multiple k-mer De Bruijn graph


### Save data

In [32]:
global_pipeline_metrics_df.to_csv("Tables/Results/Global metrics.csv")

### Global stats per assembler - per sample

#### Values

In [81]:
best_stats_dfs = []

for sample in global_pipeline_metrics_df['sample'].unique():
    print(sample)
    stats_per_assembler = pd.DataFrame()
    best_stats_per_assembler = pd.DataFrame()
    for assembler in global_pipeline_metrics_df['assembler'].unique():
        
        row = {'Assembler': assembler, 
             'Type':global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].type.unique()[0],
             'Algorithm': global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        
        best_row = {'Assembler': assembler}
        
        describe_df = global_pipeline_metrics_df[(global_pipeline_metrics_df['assembler'] == assembler) & (global_pipeline_metrics_df['sample'] == sample)].describe()

        for column in describe_df.columns:
            mean = describe_df.loc['mean',column]
            minimum = describe_df.loc['min',column]
            maximum = describe_df.loc['max',column]
            
            row[column] = "{} [{};{}]".format(round(mean, 2), int(minimum), int(maximum))
            
            if column in best_min:
                best_row[column] = minimum
            else:
                best_row[column] = maximum
                
        stats_per_assembler=stats_per_assembler.append(row, ignore_index=True)
        best_stats_per_assembler=best_stats_per_assembler.append(best_row, ignore_index=True)
    
    stats_per_assembler = stats_per_assembler.set_index(['Assembler','Algorithm', 'Type'])
    best_stats_per_assembler = best_stats_per_assembler.set_index('Assembler')
    display(stats_per_assembler)
    display(best_stats_per_assembler)
    stats_per_assembler.to_csv("Tables/Results/Global metrics per assembler - {}.csv".format(sample))
    best_stats_per_assembler.to_csv("Tables/Results/Best Global metrics per assembler - {}.csv".format(sample))
    best_stats_dfs.append([sample, best_stats_per_assembler])

LNN


,,,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,Algorithm,Type,,,,,,,,,,,,,
ABySS,Single k-mer De Bruijn graph,Genomic,1345.67 [1345;1347],20132373.0 [20132244;20132631],44619.67 [44619;44621],400.0 [400;400],13573771.33 [13573734;13573790],1211.33 [1211;1212],98.9 [98;98],192306.0 [192306;192306],99.85 [99;99],851314.0 [851314;851314],0.0 [0;0],0.0 [0;0],9569.0 [9569;9569]
BCALM2,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],19635512.0 [19635512;19635512],44825.0 [44825;44825],0.0 [0;0],13397815.0 [13397815;13397815],784.0 [784;784],97.22 [97;97],31436.0 [31436;31436],98.31 [98;98],163543.0 [163543;163543],0.0 [0;0],0.0 [0;0],18682.0 [18682;18682]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],13911109.33 [13910838;13911245],693.0 [693;693],0.0 [0;0],13658336.0 [13658336;13658336],219.0 [219;219],98.81 [98;98],140561.0 [140561;140561],99.72 [99;99],632215.0 [632215;632215],1.0 [1;1],3.0 [3;3],136741.0 [136741;136741]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],14324862.33 [14324747;14324920],1987.0 [1987;1987],0.0 [0;0],13642314.0 [13642314;13642314],261.0 [261;261],98.95 [98;98],108043.0 [108043;108043],99.69 [99;99],585610.0 [585610;585610],2.0 [2;2],2.0 [2;2],105808.0 [105808;105808]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],14402555.0 [14402555;14402555],1842.0 [1842;1842],0.0 [0;0],13704539.0 [13704539;13704539],172.0 [172;172],99.69 [99;99],210993.0 [210993;210993],99.77 [99;99],1080013.0 [1080013;1080013],1.0 [1;1],2.0 [2;2],203548.0 [203548;203548]
MetaHipMer2,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],15514698.0 [15514698;15514698],6700.0 [6700;6700],0.0 [0;0],13692577.0 [13692577;13692577],139.0 [139;139],99.59 [99;99],239185.0 [239185;239185],99.8 [99;99],850099.0 [850099;850099],1.0 [1;1],3.0 [3;3],220609.0 [220609;220609]
MINIA,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],16401048.67 [16401028;16401090],15000.33 [15000;15001],0.0 [0;0],13501499.33 [13501498;13501500],762.0 [762;762],97.53 [97;97],33550.0 [33550;33550],99.01 [99;99],177038.0 [177038;177038],2.0 [2;2],4.0 [4;4],26474.0 [26474;26474]
SKESA,Multiple k-mer De Bruijn graph,Genomic,0.0 [0;0],8202941.0 [8202941;8202941],9514.0 [9514;9514],0.0 [0;0],4470092.0 [4470092;4470092],1433.0 [1433;1433],91.77 [91;91],7385.0 [7385;7385],95.1 [95;95],59272.0 [59272;59272],12.0 [12;12],23.0 [23;23],1155.0 [1155;1155]


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,1345.0,20132631.0,44619.0,400.0,13573790.0,1211.0,98.901625,192306.0,99.851429,851314.0,0.0,0.0,9569.0
BCALM2,0.0,19635512.0,44825.0,0.0,13397815.0,784.0,97.222124,31436.0,98.307446,163543.0,0.0,0.0,18682.0
GATBMiniaPipeline,0.0,13911245.0,693.0,0.0,13658336.0,219.0,98.807198,140561.0,99.719684,632215.0,1.0,3.0,136741.0
IDBA-UD,0.0,14324920.0,1987.0,0.0,13642314.0,261.0,98.951945,108043.0,99.694880,585610.0,2.0,2.0,105808.0
MEGAHIT,0.0,14402555.0,1842.0,0.0,13704539.0,172.0,99.686868,210993.0,99.765146,1080013.0,1.0,2.0,203548.0
MetaHipMer2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
metaSPAdes,0.0,15514698.0,6700.0,0.0,13692577.0,139.0,99.591855,239185.0,99.798003,850099.0,1.0,3.0,220609.0
MINIA,0.0,16401090.0,15000.0,0.0,13501500.0,762.0,97.535204,33550.0,99.011021,177038.0,2.0,4.0,26474.0
SKESA,0.0,8202941.0,9514.0,0.0,4470092.0,1433.0,91.766425,7385.0,95.101330,59272.0,12.0,23.0,1155.0


ERR2935805


,,,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,Algorithm,Type,,,,,,,,,,,,,
ABySS,Single k-mer De Bruijn graph,Genomic,9.0 [9;9],78727988.67 [78727717;78728532],677855.33 [677853;677860],0.0 [0;0],3150980.0 [3150980;3150980],356.0 [356;356],79.03 [79;79],18568.0 [18568;18568],91.83 [91;91],88665.0 [88665;88665],0.0 [0;0],0.0 [0;0],101.0 [101;101]
BCALM2,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],315567452.0 [315567452;315567452],8035706.0 [8035706;8035706],0.0 [0;0],657074.0 [657074;657074],504.0 [504;504],0.1 [0;0],1262.0 [1262;1262],1.43 [1;1],3093.0 [3093;3093],0.0 [0;0],0.0 [0;0],35.0 [35;35]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],17140207.33 [17126362;17147130],6776.33 [6776;6777],0.0 [0;0],13905380.67 [13891328;13912407],603.0 [603;603],88.9 [88;88],90393.0 [90095;90989],89.4 [89;89],542997.0 [542997;542997],0.67 [0;1],1.33 [0;2],55722.33 [55204;56759]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],23587129.0 [23586883;23587252],14155.67 [14155;14156],0.0 [0;0],17534580.0 [17534580;17534580],2725.0 [2725;2725],89.23 [89;89],113853.0 [113853;113853],89.8 [89;89],632213.0 [632213;632213],3.0 [3;3],5.0 [5;5],51161.0 [51161;51161]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],24498566.0 [24498566;24498566],14172.0 [14172;14172],0.0 [0;0],18552996.0 [18552996;18552996],3044.0 [3044;3044],89.5 [89;89],210993.0 [210993;210993],89.76 [89;89],768410.0 [768410;768410],1.0 [1;1],2.0 [2;2],82249.0 [82249;82249]
MetaHipMer2,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],15029445.33 [15029183;15029970],11970.67 [11966;11973],0.0 [0;0],10257182.33 [10253731;10264085],5033.33 [5031;5038],73.91 [73;73],2174.67 [2170;2177],84.35 [84;84],14760.0 [14760;14760],0.0 [0;0],0.0 [0;0],1515.67 [1515;1516]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],27822010.0 [27822010;27822010],26282.0 [26282;26282],0.0 [0;0],18792639.0 [18792639;18792639],3392.0 [3392;3392],89.37 [89;89],87983.0 [87983;87983],89.93 [89;89],622107.0 [622107;622107],3.0 [3;3],7.0 [7;7],3674.0 [3674;3674]
MINIA,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],20758237.33 [20757924;20758864],37287.33 [37281;37300],0.0 [0;0],13447286.0 [13446596;13448666],1464.0 [1463;1466],84.24 [84;84],18204.67 [18038;18288],87.68 [87;87],163543.0 [163543;163543],2.0 [2;2],3.0 [3;3],7485.0 [7485;7485]
SKESA,Multiple k-mer De Bruijn graph,Genomic,0.0 [0;0],8818742.0 [8818742;8818742],8822.0 [8822;8822],0.0 [0;0],5801439.0 [5801439;5801439],2889.0 [2889;2889],62.71 [62;62],2148.0 [2148;2148],78.66 [78;78],29155.0 [29155;29155],0.0 [0;0],0.0 [0;0],1431.0 [1431;1431]


...

,,,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,Algorithm,Type,,,,,,,,,,,,,
ABySS,Single k-mer De Bruijn graph,Genomic,98.67 [0;149],61139451.0 [0;91720949],331860.0 [0;497797],4.0 [0;7],20429844.33 [0;30657287],800.33 [0;1205],53.74 [0;80],35189.0 [0;52869],56.42 [0;84],198658.0 [0;298093],1.33 [0;2],3.33 [0;6],92.67 [0;139]
BCALM2,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],170402348.0 [170402348;170402348],3508016.0 [3508016;3508016],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],8.15 [8;8],994.0 [994;994],0.0 [0;0],0.0 [0;0],49.0 [49;49]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],32809889.0 [32809112;32810830],5398.33 [5397;5400],0.0 [0;0],30564350.0 [30564302;30564396],761.0 [761;761],82.23 [82;82],113162.0 [113162;113162],83.14 [83;83],768462.0 [768462;768462],1.0 [1;1],2.0 [2;2],102115.0 [102115;102115]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],39776993.0 [39773020;39779384],20182.33 [20164;20194],0.0 [0;0],31562689.67 [31562689;31562690],2654.0 [2654;2654],81.31 [81;81],29134.0 [29134;29134],83.67 [83;83],303329.0 [303329;303329],4.0 [4;4],7.0 [7;7],20993.0 [20993;20993]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],41166407.0 [41166407;41166407],20614.0 [20614;20614],0.0 [0;0],32021766.0 [32021766;32021766],1447.0 [1447;1447],82.83 [82;82],149841.0 [149841;149841],84.22 [84;84],1205839.0 [1205839;1205839],15.0 [15;15],31.67 [31;32],88308.0 [88308;88308]
MetaHipMer2,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],13179761.33 [13174826;13184199],19356.0 [19351;19361],0.0 [0;0],1600962.67 [1599587;1601911],1309.0 [1308;1310],3.96 [3;3],1176.0 [1175;1178],25.5 [25;25],2964.0 [2964;2964],0.0 [0;0],0.0 [0;0],658.67 [658;659]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],48964620.0 [48964620;48964620],61644.0 [61644;61644],0.0 [0;0],31967753.0 [31967753;31967753],1689.0 [1689;1689],82.04 [82;82],95101.0 [95101;95101],84.33 [84;84],473277.0 [473277;473277],2.0 [2;2],3.0 [3;3],42332.0 [42332;42332]
MINIA,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],41186649.33 [41185696;41187744],67991.0 [67974;68012],0.0 [0;0],28277982.0 [28276538;28279162],3980.33 [3978;3983],74.36 [74;74],15886.33 [15817;15930],80.65 [80;80],117945.0 [117945;117945],0.0 [0;0],0.0 [0;0],7090.67 [7060;7106]
SKESA,Multiple k-mer De Bruijn graph,Genomic,0.0 [0;0],30430077.0 [30430077;30430077],1352.0 [1352;1352],0.0 [0;0],30211740.0 [30211740;30211740],846.0 [846;846],80.51 [80;80],77870.0 [77870;77870],81.25 [81;81],446163.0 [446163;446163],0.0 [0;0],0.0 [0;0],76924.0 [76924;76924]


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,0.0,91720949.0,0.0,0.0,30657287.0,0.0,80.631071,52869.0,84.624746,298093.0,0.0,0.0,139.0
BCALM2,0.0,170402348.0,3508016.0,0.0,0.0,0.0,0.000000,0.0,8.149572,994.0,0.0,0.0,49.0
GATBMiniaPipeline,0.0,32810830.0,5397.0,0.0,30564396.0,761.0,82.225440,113162.0,83.144413,768462.0,1.0,2.0,102115.0
IDBA-UD,0.0,39779384.0,20164.0,0.0,31562690.0,2654.0,81.309318,29134.0,83.671484,303329.0,4.0,7.0,20993.0
MEGAHIT,0.0,41166407.0,20614.0,0.0,32021766.0,1447.0,82.834929,149841.0,84.223732,1205839.0,15.0,31.0,88308.0
MetaHipMer2,0.0,13184199.0,19351.0,0.0,1601911.0,1308.0,3.970853,1178.0,25.512982,2964.0,0.0,0.0,659.0
metaSPAdes,0.0,48964620.0,61644.0,0.0,31967753.0,1689.0,82.040817,95101.0,84.326108,473277.0,2.0,3.0,42332.0
MINIA,0.0,41187744.0,67974.0,0.0,28279162.0,3978.0,74.366709,15930.0,80.648560,117945.0,0.0,0.0,7106.0
SKESA,0.0,30430077.0,1352.0,0.0,30211740.0,846.0,80.508554,77870.0,81.254167,446163.0,0.0,0.0,76924.0


ENN


,,,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,Algorithm,Type,,,,,,,,,,,,,
ABySS,Single k-mer De Bruijn graph,Genomic,3881.0 [3879;3884],31510320.33 [31503127;31513997],4772.33 [4769;4778],2328.0 [2328;2328],30742562.33 [30734590;30746773],1823.33 [1822;1825],98.92 [98;98],89282.0 [89282;89282],99.79 [99;99],952617.33 [952616;952618],7.33 [7;8],12.67 [12;14],86573.0 [86573;86573]
BCALM2,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],31047581.0 [31047581;31047581],25377.0 [25377;25377],0.0 [0;0],28174748.0 [28174748;28174748],3575.0 [3575;3575],92.59 [92;92],20700.0 [20700;20700],96.52 [96;96],161812.0 [161812;161812],0.0 [0;0],0.0 [0;0],17395.0 [17395;17395]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],30440762.0 [30440762;30440762],1100.0 [1100;1100],0.0 [0;0],30273320.0 [30273320;30273320],814.0 [814;814],98.46 [98;98],90988.0 [90988;90988],99.34 [99;99],632215.0 [632215;632215],3.0 [3;3],6.0 [6;6],90346.0 [90346;90346]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],30390566.0 [30390566;30390566],1822.0 [1822;1822],0.0 [0;0],30130413.0 [30130413;30130413],1279.0 [1279;1279],97.92 [97;97],65108.0 [65108;65108],99.34 [99;99],585610.0 [585610;585610],17.33 [17;18],34.67 [34;36],63725.0 [63725;63725]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],30545419.0 [30545419;30545419],814.0 [814;814],0.0 [0;0],30420265.0 [30420265;30420265],525.0 [525;525],99.39 [99;99],179870.0 [179870;179870],99.69 [99;99],632274.0 [632274;632274],9.0 [9;9],18.0 [18;18],178381.0 [178381;178381]
MetaHipMer2,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],30378682.33 [30378611;30378729],882.67 [882;883],0.0 [0;0],30264906.33 [30264835;30264953],718.67 [718;719],97.95 [97;97],102049.0 [102049;102049],98.73 [98;98],468359.0 [468359;468359],0.0 [0;0],0.0 [0;0],99990.0 [99990;99990]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],30453778.0 [30453778;30453778],640.0 [640;640],0.0 [0;0],30383186.0 [30383186;30383186],359.0 [359;359],98.92 [98;98],238611.0 [238611;238611],99.53 [99;99],1036942.0 [1036942;1036942],2.0 [2;2],6.0 [6;6],238611.0 [238611;238611]
MINIA,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],30641342.33 [30641333;30641357],16570.0 [16570;16570],0.0 [0;0],28205937.67 [28205937;28205939],3589.0 [3589;3589],92.93 [92;92],20628.0 [20628;20628],97.19 [97;97],161812.0 [161812;161812],0.0 [0;0],0.0 [0;0],17820.0 [17820;17820]
SKESA,Multiple k-mer De Bruijn graph,Genomic,0.0 [0;0],30110758.0 [30110758;30110758],2322.0 [2322;2322],0.0 [0;0],29835112.0 [29835112;29835112],1726.0 [1726;1726],97.42 [97;97],43168.0 [43168;43168],98.31 [98;98],630384.0 [630384;630384],10.0 [10;10],19.0 [19;19],42115.0 [42115;42115]


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,3879.0,31513997.0,4769.0,2328.0,30746773.0,1822.0,98.923812,89282.0,99.787690,952618.0,7.0,12.0,86573.0
BCALM2,0.0,31047581.0,25377.0,0.0,28174748.0,3575.0,92.594028,20700.0,96.523967,161812.0,0.0,0.0,17395.0
GATBMiniaPipeline,0.0,30440762.0,1100.0,0.0,30273320.0,814.0,98.462815,90988.0,99.340345,632215.0,3.0,6.0,90346.0
IDBA-UD,0.0,30390566.0,1822.0,0.0,30130413.0,1279.0,97.922883,65108.0,99.343697,585610.0,17.0,34.0,63725.0
MEGAHIT,0.0,30545419.0,814.0,0.0,30420265.0,525.0,99.388110,179870.0,99.693845,632274.0,9.0,18.0,178381.0
MetaHipMer2,0.0,30378729.0,882.0,0.0,30264953.0,718.0,97.946876,102049.0,98.726583,468359.0,0.0,0.0,99990.0
metaSPAdes,0.0,30453778.0,640.0,0.0,30383186.0,359.0,98.917250,238611.0,99.530051,1036942.0,2.0,6.0,238611.0
MINIA,0.0,30641357.0,16570.0,0.0,28205939.0,3589.0,92.933709,20628.0,97.188180,161812.0,0.0,0.0,17820.0
SKESA,0.0,30110758.0,2322.0,0.0,29835112.0,1726.0,97.424467,43168.0,98.313391,630384.0,10.0,19.0,42115.0


#### rank

In [82]:
rank_dfs = []
for sample,df in best_stats_dfs:
    print(sample)
    #display(df)
    
    rank_df = pd.DataFrame(index=df.index)
    
    for column in df.columns:
        sorted_order = True if column in best_min else False
        sorted_column = list(df[column].sort_values(ascending=sorted_order))
        
        rank_list = []
        for item in df[column]:
            if item == sorted_column[0]:
                rank = 1
            else:
                rank = sorted_column.index(item) + 1
            rank_list.append(rank)
        rank_df[column] = rank_list 
    display(rank_df)
    rank_df.to_csv("Tables/Results/Rank Global metrics per assembler - {}.csv".format(sample))
    rank_dfs.append([sample, rank_df])
            

LNN


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,11,1,11,12,7,11,5,5,1,2,1,1,10
BCALM2,1,3,12,1,9,10,10,10,10,10,1,1,9
GATBMiniaPipeline,1,9,3,1,4,6,6,6,4,7,7,9,4
IDBA-UD,1,8,5,1,5,8,4,8,6,8,10,7,7
MEGAHIT,1,7,4,1,1,4,1,3,3,1,7,7,3
MetaHipMer2,1,12,1,1,12,1,12,12,12,12,1,1,12
metaSPAdes,1,6,6,1,2,3,2,2,2,3,7,9,2
MINIA,1,4,9,1,8,9,9,9,8,9,10,11,8
SKESA,1,11,8,1,11,12,11,11,11,11,12,12,11


ERR2935805


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,12,2,11,1,10,3,7,7,1,8,1,1,10
BCALM2,1,1,12,1,11,4,11,11,11,11,1,1,12
GATBMiniaPipeline,1,9,2,1,5,5,4,4,5,6,1,1,3
IDBA-UD,1,7,5,1,4,7,3,3,3,4,10,10,4
MEGAHIT,1,6,6,1,3,9,1,2,4,3,8,8,2
MetaHipMer2,1,10,4,1,7,12,8,9,9,10,1,1,8
metaSPAdes,1,5,7,1,2,10,2,5,2,5,10,11,6
MINIA,1,8,9,1,6,6,6,8,7,7,9,9,5
SKESA,1,12,3,1,9,8,9,10,10,9,1,1,9


LHS


...

,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,11,2,11,11,1,4,5,5,6,5,8,7,5
BCALM2,1,1,12,1,12,12,12,12,12,12,1,1,12
GATBMiniaPipeline,1,7,4,1,5,6,4,6,2,7,6,6,6
IDBA-UD,1,9,7,1,8,9,7,9,5,10,12,12,9
MEGAHIT,1,5,5,1,2,5,1,4,1,6,11,11,4
MetaHipMer2,1,10,3,1,6,7,8,7,8,9,1,1,7
metaSPAdes,1,8,2,1,3,2,2,3,3,2,6,7,2
MINIA,1,4,10,1,11,11,11,11,10,11,1,1,11
SKESA,1,12,6,1,9,8,9,8,9,8,10,10,8


ERR2984773


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,1,2,1,1,5,1,7,7,1,8,1,1,10
BCALM2,1,1,12,1,12,1,12,12,11,12,1,1,12
GATBMiniaPipeline,1,9,4,1,6,5,2,3,6,3,7,7,2
IDBA-UD,1,8,6,1,4,11,5,8,5,7,10,10,7
MEGAHIT,1,7,7,1,2,8,1,2,3,1,12,12,3
MetaHipMer2,1,12,5,1,10,7,10,10,10,10,1,1,9
metaSPAdes,1,4,9,1,3,9,3,5,2,5,8,8,6
MINIA,1,6,10,1,9,12,9,9,9,9,1,1,8
SKESA,1,11,3,1,8,6,8,6,8,6,1,1,4


ENN


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,11,1,9,11,1,9,2,7,1,2,8,9,7
BCALM2,1,2,12,1,12,11,12,11,12,11,1,1,12
GATBMiniaPipeline,1,8,5,1,5,6,5,6,6,7,6,5,6
IDBA-UD,1,9,7,1,7,7,8,8,5,9,12,12,8
MEGAHIT,1,4,2,1,2,4,1,4,2,6,10,10,4
MetaHipMer2,1,10,3,1,6,5,6,5,8,10,1,1,5
metaSPAdes,1,6,1,1,3,1,3,1,3,1,5,5,1
MINIA,1,3,11,1,11,12,11,12,10,11,1,1,11
SKESA,1,12,8,1,9,8,9,9,9,8,11,11,9


#### radar plots

In [107]:
import plotly.graph_objects as go

for sample, df in rank_dfs:
    print(sample)

    categories = df.columns
    categories_original =  ['Ns', 'basepairs', 'contigs', 'mapped_reads', 'max_contig', 'n50'] 
    categories_filtered = ['filtered_Ns', 'filtered_basepairs', 'filtered_contigs', 'filtered_mapped_reads', 'filtered_n50', 'misassembled contigs', 'misassembly events']
    
    fig = go.Figure()
    fig_original = go.Figure()
    fig_filtered = go.Figure()
    
    for assembler, row in df.iterrows():
        row_original = []
        row_filtered = []
        
        for col in categories_original:
            row_original.append(row.loc[:].at[col])
            
        for col in categories_filtered:
            row_filtered.append(row.loc[:].at[col])

        fig.add_trace(go.Scatterpolar(
          r=list(row),
          theta=categories,
          name=assembler))

        fig_original.add_trace(go.Scatterpolar(
          r=row_original,
          theta=categories_original,
          name=assembler))
        
        fig_filtered.add_trace(go.Scatterpolar(
          r=row_filtered,
          theta=categories_filtered,
          name=assembler))
    
    fig.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
          range=[1, 13],
            autorange="reversed"
        )),
       title=sample
    )
    
    fig_original.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
          range=[1, 13],
            autorange="reversed"
        )),
       title="{} Original Assembly".format(sample)
    )
    
    fig_filtered.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
          range=[1, 13],
            autorange="reversed"
        )),
       title="{} Filtered Assembly".format(sample)
    )

    fig.show()
    fig_original.show()
    fig_filtered.show()


LNN


ERR2935805


LHS


EMS


ERR2984773


ENN


### Global statistics per assembler type

In [15]:
stats_per_assembler_type = pd.DataFrame()
for assembly_type in global_pipeline_metrics_df.type.unique():
    row={'Assembler Type': assembly_type}
    lala=global_pipeline_metrics_df[(global_pipeline_metrics_df.type == assembly_type) & (global_pipeline_metrics_df.algorythm == "Multiple k-mer De Bruijn graph")].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2), int(minimum), int(maximum))
    stats_per_assembler_type=stats_per_assembler_type.append(row, ignore_index=True)
stats_per_assembler_type = stats_per_assembler_type.set_index('Assembler Type')
stats_per_assembler_type.to_csv("Tables/Results/Global metrics per assembler type multiple kmer.csv")

In [16]:
stats_per_assembler_type

,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler Type,,,,,,,,,,,,,
Metagenomic,3659.67 [0;198470],26973161.17 [0;91720949],64807.29 [0;677860],825.3 [0;27980],18803824.92 [0;32021766],1103.4 [0;5038],77.08 [0;99],99919.05 [0;239185],79.25 [0;99],553667.94 [0;1205839],3.06 [0;18],6.01 [0;36],80396.02 [0;238611]
Genomic,0.0 [0;0],22538873.06 [2957060;46535940],8034.89 [36;39819],0.0 [0;0],20457526.67 [2950393;32500826],1064.67 [21;4999],88.55 [57;99],148216.83 [2148;351027],92.91 [78;99],677496.0 [29155;1303875],34.61 [0;572],69.33 [0;1148],139740.44 [1155;351027]


### Global stats per assembly algorithm

In [17]:
stats_per_assembler_algo = pd.DataFrame()
for assembly_algo in global_pipeline_metrics_df.algorythm.unique():
    row={'Assembler Algorithm': assembly_algo}
    lala=global_pipeline_metrics_df[global_pipeline_metrics_df.algorythm == assembly_algo].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2), int(minimum), int(maximum))
    stats_per_assembler_algo=stats_per_assembler_algo.append(row, ignore_index=True)
stats_per_assembler_algo = stats_per_assembler_algo.set_index('Assembler Algorithm')
stats_per_assembler_algo.to_csv("Tables/Results/Global metrics per assembler algorithm.csv")

In [18]:
stats_per_assembler_algo

,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler Algorithm,,,,,,,,,,,,,
Multiple k-mer De Bruijn graph,2561.77 [0;198470],25642874.73 [0;91720949],47775.57 [0;677860],577.71 [0;27980],19299935.44 [0;32500826],1091.78 [0;5038],80.52 [0;99],114408.38 [0;351027],83.35 [0;99],590816.36 [0;1303875],12.53 [0;572],25.01 [0;1148],98199.34 [0;351027]
Single k-mer De Bruijn graph,0.0 [0;0],91329784.0 [15334799;364761201],1685021.14 [15000;8175890],0.0 [0;0],15268485.33 [0;28279162],2222.08 [0;6250],61.68 [0;97],14762.42 [0;33550],68.97 [1;99],109530.67 [994;177038],0.33 [0;2],0.58 [0;4],10118.22 [35;26474]


### Global stats per distribution

In [13]:
for distribution in global_pipeline_metrics_df.distribution.unique():
    stats_per_distribution = pd.DataFrame()
    for assembler in global_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':global_pipeline_metrics_df[(global_pipeline_metrics_df.assembler == assembler) & (global_pipeline_metrics_df.distribution == distribution)].type.unique()[0],
             'Algorithm': global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=global_pipeline_metrics_df[(global_pipeline_metrics_df.assembler == assembler) & (global_pipeline_metrics_df.distribution == distribution)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            minimum = lala.loc['min',column]
            maximum = lala.loc['max',column]
            row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
        stats_per_distribution=stats_per_distribution.append(row, ignore_index=True)
    stats_per_distribution = stats_per_distribution.set_index('Assembler')
    stats_per_distribution.to_csv("Tables/Results/{} - Global metrics per assembler.csv".format(distribution))
    print(distribution)
    display(stats_per_distribution)

Log


,Algorithm,Ns,Type,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,678.5 [9;1349],Metagenomic,49430721.75 [20132631;78728532],361241.5 [44621;677860],200.0 [0;400],8362124.25 [3150980;13573734],784.0 [356;1212],89.57 [79;99],105437.0 [18568;192306],199.54 [101;297],469989.5 [88665;851314],0.0 [0;0],0.0 [0;0],4835.0 [101;9569]
BCALM2,Single k-mer De Bruijn graph,0.0 [0;0],Genomic,233321388.33 [19635512;364761201],5418807.0 [44825;8175890],0.0 [0;0],4688952.0 [11967;13397815],433.0 [11;784],32.44 [0;97],11253.33 [1062;31436],36.14 [1;98],55949.33 [1212;163543],0.0 [0;0],0.0 [0;0],6255.33 [35;18682]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,14892063.33 [13638625;17126542],2819.5 [693;6777],0.0 [0;0],13685813.83 [13507672;13891396],527.83 [219;762],95.04 [89;98],107608.33 [90095;140561],95.72 [89;100],602475.67 [542997;632215],1.0 [0;2],3.0 [0;6],94548.33 [55075;136741]
IDBA-UD,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,17234505.33 [13791669;23587129],5722.33 [1025;14155],0.0 [0;0],14926006.67 [13601126;17534580],1184.67 [261;2725],95.23 [89;99],109414.33 [106347;113853],95.91 [90;99],616678.67 [585610;632213],2.33 [2;3],4.0 [2;5],85565.0 [51161;105808]
MEGAHIT,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,17581572.33 [13843596;24498566],5593.67 [767;14172],0.0 [0;0],15309833.0 [13671964;18552996],1181.0 [172;3044],97.05 [93;100],197842.0 [171540;210993],97.63 [94;100],826891.0 [632250;1080013],0.67 [0;1],1.33 [0;2],152385.33 [82249;203548]
MetaHipMer2,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,15030376.0 [15029970;15030782],11970.5 [11966;11975],0.0 [0;0],10265689.0 [10264085;10267293],5041.0 [5038;5044],74.37 [74;74],2173.5 [2170;2177],85.27 [85;85],14760.0 [14760;14760],0.0 [0;0],0.0 [0;0],1513.0 [1511;1515]
metaSPAdes,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,19164672.33 [14157309;27822010],11693.67 [2099;26282],0.0 [0;0],15386033.67 [13672885;18792639],1238.33 [139;3392],95.81 [89;99],174651.33 [87983;239185],96.17 [90;100],796644.67 [622107;917728],1.67 [1;3],4.33 [3;7],139179.67 [3674;220609]
MINIA,Single k-mer De Bruijn graph,0.0 [0;0],Genomic,17498121.83 [15334712;20758864],25649.33 [14997;37300],0.0 [0;0],13290066.17 [12920975;13501499],1465.67 [762;2172],87.93 [81;97],20765.0 [10582;33550],92.64 [87;99],168041.33 [163543;177038],1.33 [0;2],2.33 [0;4],13902.0 [7485;26474]
SKESA,Multiple k-mer De Bruijn graph,0.0 [0;0],Genomic,6659581.0 [2957060;8818742],6124.0 [36;9514],0.0 [0;0],4407308.0 [2950393;5801439],1447.67 [21;2889],82.19 [62;92],120186.67 [2148;351027],88.7 [78;95],240146.67 [29155;632013],4.0 [0;12],7.67 [0;23],117871.0 [1155;351027]


Even


,Algorithm,Ns,Type,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,1823.12 [147;3884],Metagenomic,47705094.62 [31503127;91720949],138398.25 [4769;497797],1103.75 [5;2328],30810219.75 [30632246;30988200],1170.62 [497;1825],100.22 [81;107],98691.38 [52698;138706],103.33 [92;108],747140.75 [297881;952618],4.0 [2;8],7.12 [3;14],79091.88 [139;124246]
BCALM2,Single k-mer De Bruijn graph,0.0 [0;0],Genomic,80340494.0 [31047581;170402348],1261858.33 [25377;3508016],0.0 [0;0],14860681.0 [0;28174748],3275.0 [0;6250],39.88 [0;92],7918.67 [0;20700],56.66 [8;96],66942.33 [994;161812],0.0 [0;0],0.0 [0;0],6029.67 [49;17395]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,31253352.33 [30440762;32810830],2480.44 [943;5400],0.0 [0;0],30393672.67 [30273320;30564396],744.0 [657;814],93.59 [82;98],108898.33 [90988;122545],94.75 [84;100],723087.33 [632215;768585],1.67 [1;3],3.0 [1;6],104806.0 [90346;121957]
IDBA-UD,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,33525497.0 [30390566;39779384],7944.44 [1822;20194],0.0 [0;0],30613010.56 [30130413;31562690],1729.0 [1254;2654],92.47 [81;98],52473.33 [29134;65108],94.54 [84;100],452420.67 [303329;585610],11.44 [4;18],21.22 [7;36],48829.89 [20993;63725]
MEGAHIT,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,34103641.33 [30545419;41166407],7469.0 [814;20614],0.0 [0;0],30953572.33 [30418624;32021766],830.0 [518;1447],94.89 [84;100],167083.67 [149841;179870],96.22 [86;101],868917.67 [632274;1205839],11.33 [9;15],23.56 [18;32],146076.33 [88308;178381]
MetaHipMer2,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,24646614.11 [13174826;30381792],7041.0 [882;19361],0.0 [0;0],20709564.33 [1599587;30264953],913.22 [712;1310],66.63 [3;98],69954.0 [1175;106637],74.44 [25;99],313232.67 [2964;468375],0.0 [0;0],0.0 [0;0],69010.56 [658;106383]
metaSPAdes,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,36633480.0 [30453778;48964620],21015.0 [640;61644],0.0 [0;0],30917818.67 [30383186;31967753],804.33 [359;1689],94.17 [83;100],180893.67 [95101;238611],95.36 [85;100],809315.0 [473277;1036942],1.67 [1;2],4.0 [3;6],163304.0 [42332;238611]
MINIA,Single k-mer De Bruijn graph,0.0 [0;0],Genomic,34159156.67 [30641333;41187744],33769.78 [16570;68012],0.0 [0;0],28234093.56 [28205937;28279162],3714.44 [3574;3983],86.52 [74;92],19098.78 [15817;20782],91.29 [80;97],147189.67 [117945;161812],0.0 [0;0],0.0 [0;0],14292.56 [7060;17967]
SKESA,Multiple k-mer De Bruijn graph,0.0 [0;0],Genomic,30263322.33 [30110758;30430077],1647.67 [1269;2322],0.0 [0;0],30037221.0 [29835112;30211740],1154.33 [846;1726],91.9 [80;97],65058.67 [43168;77870],92.76 [81;98],569520.0 [446163;632013],6.33 [0;10],12.33 [0;19],64292.67 [42115;76924]


## Reference-based metrics

### Load data

In [14]:
report_glob = glob.glob('../Results/*/*/report/pipeline_report_tables.json')
reference_pipeline_metrics_df = pd.DataFrame()

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for reference, data in json_report[sample]['ReferenceTables'].items():
                for row in data:
                    for item in row:
                        reference_pipeline_metrics_df = reference_pipeline_metrics_df.append({'run': stats_run,
                                                                                        'sample': sample,
                                                                                        'assembler': item['assembler'],
                                                                                        'reference': reference,
                                                                                        'LSA': item['contiguity'],
                                                                                        'breadth_of_coverage': item['breadth_of_coverage'],
                                                                                        'multiplicity': item['multiplicity'],
                                                                                        'validity': item['validity'],
                                                                                        'parsimony': item['parsimony'],
                                                                                        'identity': item['identity'],
                                                                                        'lowest_identity': item['lowest_identity'],
                                                                                        'L90': item['L90'],
                                                                                        'contigs': item['aligned_contigs'],
                                                                                        'NA50': item['NA50'],
                                                                                        'NG50': item['NG50'],
                                                                                        'basepairs': item['aligned_basepairs'],
                                                                                        'Ns': item['Ns'],
                                                                                        'misassembled contigs': item['misassembled_contigs'],
                                                                                        'misassembly events': item['misassembly_events']},
                                                                                       ignore_index=True)

reference_pipeline_metrics_df['distribution'] = np.where(reference_pipeline_metrics_df['sample'].isin(log_distributed), 'Log', 'Even')
reference_pipeline_metrics_df['type'] = np.where(reference_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
reference_pipeline_metrics_df['algorythm'] = np.where(reference_pipeline_metrics_df['assembler'].isin(single_kmer), 'Single k-mer De Bruijn graph', 'Multiple k-mer De Bruijn graph')
reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassembled contigs', 'misassembly events', 'multiplicity','validity','parsimony','identity','lowest_identity']] = reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassembled contigs', 'misassembly events','multiplicity','validity','parsimony','identity','lowest_identity']].apply(pd.to_numeric)
display(reference_pipeline_metrics_df)

Processing pipeline_report_tables.json data from run2...
Processing pipeline_report_tables.json data from run1...
Processing pipeline_report_tables.json data from run3...
Processing pipeline_report_tables.json data from run2...


,L90,LSA,NA50,NG50,Ns,assembler,basepairs,breadth_of_coverage,contigs,identity,...,misassembly events,multiplicity,parsimony,reference,run,sample,validity,distribution,type,algorythm
0,0.0,0.000000,0.0,0.0,0.0,ABySS,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,Bacillus_subtilis,run2,ERR2935805,0.000000,Log,Metagenomic,Multiple k-mer De Bruijn graph
1,0.0,0.000765,1271.0,0.0,0.0,BCALM2,647119.0,0.159953,496.0,0.999942,...,0.0,1.000771,1.000825,Bacillus_subtilis,run2,ERR2935805,0.999946,Log,Genomic,Single k-mer De Bruijn graph
2,132.0,0.028610,31048.0,30028.0,0.0,GATBMiniaPipeline,3960148.0,0.978859,205.0,0.999846,...,0.0,1.001038,1.001106,Bacillus_subtilis,run2,ERR2935805,0.999932,Log,Metagenomic,Multiple k-mer De Bruijn graph
3,41.0,0.057949,101732.0,101732.0,0.0,IDBA-UD,3973333.0,0.982118,74.0,0.999048,...,2.0,1.000234,1.000349,Bacillus_subtilis,run2,ERR2935805,0.999884,Log,Metagenomic,Multiple k-mer De Bruijn graph
4,13.0,0.189992,263503.0,263503.0,0.0,MEGAHIT,3982262.0,0.984325,27.0,0.998894,...,0.0,0.999808,1.000243,Bacillus_subtilis,run2,ERR2935805,0.999566,Log,Metagenomic,Multiple k-mer De Bruijn graph
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2071,0.0,0.000000,0.0,0.0,0.0,MINIA,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,Staphylococcus_aureus_plasmid3,run2,EMS,0.000000,Even,Genomic,Single k-mer De Bruijn graph
2072,0.0,0.908119,2718.0,2718.0,0.0,SKESA,2718.0,0.908119,1.0,1.000000,...,0.0,1.000000,1.000000,Staphylococcus_aureus_plasmid3,run2,EMS,1.000000,Even,Genomic,Multiple k-mer De Bruijn graph
2073,0.0,0.999666,2992.0,2992.0,0.0,SPAdes,2992.0,0.999666,1.0,1.000000,...,0.0,1.000000,1.000000,Staphylococcus_aureus_plasmid3,run2,EMS,1.000000,Even,Genomic,Multiple k-mer De Bruijn graph
2074,0.0,0.964250,2886.0,2886.0,0.0,Unicycler,2886.0,0.964250,1.0,1.000000,...,0.0,1.000000,1.000000,Staphylococcus_aureus_plasmid3,run2,EMS,1.000000,Even,Genomic,Multiple k-mer De Bruijn graph


In [15]:
reference_pipeline_metrics_df.dtypes

L90                     float64
LSA                     float64
NA50                    float64
NG50                    float64
Ns                      float64
assembler                object
basepairs               float64
breadth_of_coverage     float64
contigs                 float64
identity                float64
lowest_identity         float64
misassembled contigs    float64
misassembly events      float64
multiplicity            float64
parsimony               float64
reference                object
run                      object
sample                   object
validity                float64
distribution             object
type                     object
algorythm                object
dtype: object

### Save data

In [16]:
reference_pipeline_metrics_df.to_csv("Tables/Results/Reference metrics.csv")

### Reference statistics per assembler type

In [17]:
ref_stats_per_assembler_type = pd.DataFrame()
for assembly_type in reference_pipeline_metrics_df.type.unique():
    row={'Assembler Type': assembly_type}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.type == assembly_type].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    ref_stats_per_assembler_type=ref_stats_per_assembler_type.append(row, ignore_index=True)
ref_stats_per_assembler_type = ref_stats_per_assembler_type.set_index('Assembler Type')
ref_stats_per_assembler_type.to_csv("Tables/Results/Reference metrics per assembler type.csv")

In [18]:
ref_stats_per_assembler_type

,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler Type,,,,,,,,,,,,,,,
Metagenomic,28.13 [0;1033],0.18 [0;1],70927.54 [0;593167],70320.34 [0;593167],70.54 [0;12098],1790676.74 [0;6781605],0.62 [0;1],81.73 [0;2404],0.72 [0;1],0.69 [0;1],0.32 [0;9],0.6 [0;15],0.73 [0;1],0.74 [0;5],0.72 [0;1]
Genomic,37.39 [0;921],0.13 [0;1],54309.38 [0;475179],53317.35 [0;475179],0.0 [0;0],1647148.01 [0;6728888],0.53 [0;1],121.73 [0;1877],0.63 [0;1],0.6 [0;1],1.47 [0;565],2.9 [0;1134],0.63 [0;1],0.64 [0;1],0.63 [0;1]


### Reference stats per assembly algorithm

In [19]:
ref_stats_per_assembler_algo = pd.DataFrame()
for assembly_algo in reference_pipeline_metrics_df.algorythm.unique():
    row={'Assembler Algorithm': assembly_algo}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.algorythm == assembly_algo].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    ref_stats_per_assembler_algo=ref_stats_per_assembler_algo.append(row, ignore_index=True)
ref_stats_per_assembler_algo = ref_stats_per_assembler_algo.set_index('Assembler Algorithm')
ref_stats_per_assembler_algo.to_csv("Tables/Results/Reference metrics per assembler algorithm.csv")

In [20]:
ref_stats_per_assembler_algo

,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler Algorithm,,,,,,,,,,,,,,,
Multiple k-mer De Bruijn graph,27.11 [0;1033],0.19 [0;1],75155.49 [0;593167],74317.83 [0;593167],48.34 [0;12098],1803152.21 [0;6781605],0.62 [0;1],76.78 [0;2404],0.72 [0;1],0.68 [0;1],0.98 [0;565],1.92 [0;1134],0.72 [0;1],0.73 [0;5],0.72 [0;1]
Single k-mer De Bruijn graph,56.12 [0;921],0.04 [0;0],9228.89 [0;65138],8758.16 [0;64960],0.0 [0;0],1372388.53 [0;6524118],0.4 [0;0],205.32 [0;1495],0.51 [0;1],0.5 [0;1],0.03 [0;2],0.04 [0;4],0.51 [0;1],0.51 [0;1],0.51 [0;1]


### Reference stats per assembler

In [21]:
ref_stats_per_assembler = pd.DataFrame()
for assembler in reference_pipeline_metrics_df.assembler.unique():
    row={'Assembler': assembler, 
         'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
         'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    ref_stats_per_assembler=ref_stats_per_assembler.append(row, ignore_index=True)
ref_stats_per_assembler = ref_stats_per_assembler.set_index('Assembler')
ref_stats_per_assembler.to_csv("Tables/Results/Reference metrics per assembler.csv")

In [22]:
ref_stats_per_assembler

,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,57.11 [0;1033],0.21 [0;0],76534.51 [0;351511],76400.56 [0;351511],66.98 [0;2170],Metagenomic,1920076.87 [0;6781605],0.66 [0;0],86.45 [0;1404],0.72 [0;1],0.7 [0;1],0.22 [0;8],0.4 [0;14],0.73 [0;1],0.73 [0;1],0.72 [0;1]
BCALM2,Single k-mer De Bruijn graph,36.16 [0;921],0.03 [0;0],5541.46 [0;65138],5229.32 [0;64934],0.0 [0;0],Genomic,897715.86 [0;6515102],0.26 [0;0],178.13 [0;1495],0.37 [0;1],0.37 [0;1],0.0 [0;0],0.0 [0;0],0.37 [0;1],0.37 [0;1],0.37 [0;1]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,28.24 [0;459],0.22 [0;1],80158.39 [0;452364],74423.64 [0;351186],0.0 [0;0],Metagenomic,1966705.14 [0;6716522],0.68 [0;1],50.97 [0;604],0.74 [0;1],0.73 [0;1],0.12 [0;3],0.25 [0;6],0.75 [0;1],0.75 [0;1],0.74 [0;1]
IDBA-UD,Multiple k-mer De Bruijn graph,47.83 [0;292],0.17 [0;1],52707.09 [0;475377],51305.18 [0;475377],0.0 [0;0],Metagenomic,1958660.56 [0;6710005],0.66 [0;1],84.49 [0;449],0.77 [0;1],0.68 [0;1],0.77 [0;9],1.32 [0;15],0.77 [0;1],0.83 [0;5],0.76 [0;1]
MEGAHIT,Multiple k-mer De Bruijn graph,14.35 [0;147],0.24 [0;1],110053.78 [0;543128],111658.51 [0;543128],0.0 [0;0],Metagenomic,1977154.33 [0;6738936],0.69 [0;1],33.88 [0;223],0.78 [0;1],0.73 [0;1],0.61 [0;4],1.24 [0;7],0.79 [0;1],0.8 [0;1],0.79 [0;1]
MetaHipMer2,Multiple k-mer De Bruijn graph,17.76 [0;110],0.19 [0;0],47999.15 [0;257424],47344.33 [0;257424],0.0 [0;0],Metagenomic,1559257.44 [0;6717767],0.59 [0;0],134.85 [0;2404],0.85 [0;1],0.84 [0;1],0.0 [0;0],0.0 [0;0],0.85 [0;1],0.85 [0;1],0.85 [0;1]
metaSPAdes,Multiple k-mer De Bruijn graph,14.96 [0;90],0.22 [0;1],104423.09 [0;593167],103538.27 [0;593167],0.0 [0;0],Metagenomic,1974311.24 [0;6733500],0.69 [0;1],29.81 [0;167],0.79 [0;1],0.76 [0;1],0.15 [0;2],0.36 [0;6],0.8 [0;1],0.8 [0;1],0.79 [0;1]
MINIA,Single k-mer De Bruijn graph,76.08 [0;649],0.05 [0;0],12916.32 [0;65138],12287.0 [0;64960],0.0 [0;0],Genomic,1847061.2 [0;6524118],0.54 [0;0],232.52 [0;1147],0.65 [0;1],0.63 [0;1],0.06 [0;2],0.09 [0;4],0.65 [0;1],0.65 [0;1],0.65 [0;1]
SKESA,Multiple k-mer De Bruijn graph,37.44 [0;670],0.16 [0;0],49801.04 [0;351027],49165.21 [0;351027],0.0 [0;0],Genomic,1644801.71 [0;6673601],0.58 [0;0],106.01 [0;1877],0.68 [0;1],0.6 [0;1],0.53 [0;12],0.96 [0;23],0.68 [0;1],0.68 [0;1],0.68 [0;1]


### Reference stats per reference replicon

#### Per assembler for each reference replicon

In [23]:
reference_pipeline_metrics_df.reference.unique()

array(['Bacillus_subtilis', 'Enterococcus_faecalis',
       'Escherichia_coli_plasmid', 'Escherichia_coli',
       'Lactobacillus_fermentum', 'Listeria_monocytogenes',
       'Pseudomonas_aeruginosa', 'Salmonella_enterica',
       'Staphylococcus_aureus', 'Staphylococcus_aureus_plasmid1',
       'Staphylococcus_aureus_plasmid2', 'Staphylococcus_aureus_plasmid3'],
      dtype=object)

In [24]:
for reference in reference_pipeline_metrics_df.reference.unique():
    ref_stats_per_reference_per_assembler = pd.DataFrame()
    for assembler in reference_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
             'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=reference_pipeline_metrics_df[(reference_pipeline_metrics_df.assembler == assembler) & (reference_pipeline_metrics_df.reference == reference)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            minimum = lala.loc['min',column]
            maximum = lala.loc['max',column]
            row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
        ref_stats_per_reference_per_assembler=ref_stats_per_reference_per_assembler.append(row, ignore_index=True)
    ref_stats_per_reference_per_assembler = ref_stats_per_reference_per_assembler.set_index('Assembler')
    ref_stats_per_reference_per_assembler.to_csv("Tables/Results/{} - Reference metrics per assembler.csv".format(reference))
    print(reference)
    display(ref_stats_per_reference_per_assembler)

Bacillus_subtilis


,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,185.83 [0;1033],0.07 [0;0],155739.83 [0;295233],155677.17 [0;295233],78.42 [0;400],Metagenomic,3274516.83 [0;3991152],0.81 [0;0],213.17 [0;1141],0.83 [0;0],0.83 [0;0],0.0 [0;0],0.0 [0;0],0.84 [0;1],0.84 [0;1],0.83 [0;0]
BCALM2,Single k-mer De Bruijn graph,30.4 [0;129],0.01 [0;0],18084.13 [0;65138],17635.0 [0;64934],0.0 [0;0],Genomic,1973728.93 [0;3951974],0.49 [0;0],411.53 [0;1467],0.8 [0;1],0.8 [0;1],0.0 [0;0],0.0 [0;0],0.8 [0;1],0.8 [0;1],0.8 [0;1]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,95.13 [13;459],0.11 [0;0],172788.47 [8915;269695],170206.53 [8732;263445],0.0 [0;0],Metagenomic,3964562.93 [3882084;3984103],0.98 [0;0],137.33 [27;604],1.0 [0;1],0.99 [0;1],0.4 [0;2],1.2 [0;6],1.0 [1;1],1.0 [1;1],1.0 [0;1]
IDBA-UD,Multiple k-mer De Bruijn graph,84.73 [18;292],0.05 [0;0],122499.47 [14006;252653],114309.73 [13214;212969],0.0 [0;0],Metagenomic,3968879.13 [3925882;3977845],0.98 [0;0],130.4 [34;418],1.0 [0;0],0.89 [0;0],0.53 [0;2],1.07 [0;5],1.0 [1;1],1.0 [1;1],1.0 [0;0]
MEGAHIT,Multiple k-mer De Bruijn graph,33.67 [13;147],0.12 [0;0],219465.33 [26511;294549],219633.2 [26209;294549],0.0 [0;0],Metagenomic,3984618.47 [3969756;3989518],0.98 [0;0],57.93 [27;223],0.99 [0;0],0.86 [0;0],0.73 [0;2],1.67 [0;4],1.0 [0;1],1.0 [0;1],1.0 [0;0]
MetaHipMer2,Multiple k-mer De Bruijn graph,8.45 [0;16],0.05 [0;0],141225.27 [1145;257424],140859.73 [0;257424],0.0 [0;0],Metagenomic,2865084.18 [205541;3976387],0.71 [0;0],319.82 [27;1415],1.0 [0;1],0.98 [0;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [0;1]
metaSPAdes,Multiple k-mer De Bruijn graph,29.2 [9;66],0.13 [0;0],195863.93 [61490;343843],195489.27 [62046;343843],0.0 [0;0],Metagenomic,3978593.47 [3971865;3987932],0.98 [0;0],54.0 [18;110],1.0 [0;0],0.98 [0;0],0.53 [0;2],1.6 [0;6],1.0 [0;1],1.0 [0;1],1.0 [0;1]
MINIA,Single k-mer De Bruijn graph,170.4 [66;649],0.03 [0;0],43919.73 [6662;65138],43207.8 [6313;64960],0.0 [0;0],Genomic,3932778.0 [3821823;3953200],0.97 [0;0],249.8 [116;783],1.0 [0;1],1.0 [0;1],0.53 [0;2],0.93 [0;4],1.0 [1;1],1.0 [1;1],1.0 [0;1]
SKESA,Multiple k-mer De Bruijn graph,10.2 [0;20],0.06 [0;0],145456.6 [0;269493],144190.8 [0;263164],0.0 [0;0],Genomic,2381901.8 [0;3971432],0.59 [0;0],18.0 [0;34],0.6 [0;1],0.6 [0;1],0.0 [0;0],0.0 [0;0],0.6 [0;1],0.6 [0;1],0.6 [0;1]


Enterococcus_faecalis


,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,15.42 [0;64],0.1 [0;0],127665.17 [0;241090],127665.17 [0;241090],0.0 [0;0],Metagenomic,1880638.58 [0;2828460],0.66 [0;0],28.08 [0;108],0.67 [0;0],0.66 [0;0],0.0 [0;0],0.0 [0;0],0.68 [0;1],0.68 [0;1],0.67 [0;1]
BCALM2,Single k-mer De Bruijn graph,20.87 [0;107],0.01 [0;0],6332.2 [0;30453],6084.0 [0;30452],0.0 [0;0],Genomic,596263.2 [0;2768590],0.21 [0;0],69.13 [0;180],0.4 [0;1],0.4 [0;1],0.0 [0;0],0.0 [0;0],0.4 [0;1],0.4 [0;1],0.4 [0;1]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,11.0 [0;24],0.1 [0;0],108568.4 [0;235254],108568.4 [0;235254],0.0 [0;0],Metagenomic,1679183.33 [0;2820351],0.59 [0;0],22.47 [0;46],0.6 [0;1],0.59 [0;1],0.2 [0;1],0.4 [0;2],0.6 [0;1],0.6 [0;1],0.6 [0;1]
IDBA-UD,Multiple k-mer De Bruijn graph,39.4 [0;147],0.06 [0;0],48945.4 [0;113206],50217.6 [0;118897],0.0 [0;0],Metagenomic,1679559.0 [0;2811602],0.59 [0;0],67.8 [0;231],0.6 [0;0],0.56 [0;0],0.6 [0;3],1.2 [0;6],0.6 [0;1],0.6 [0;1],0.6 [0;0]
MEGAHIT,Multiple k-mer De Bruijn graph,8.6 [0;17],0.14 [0;0],141202.6 [0;235381],141848.6 [0;238546],0.0 [0;0],Metagenomic,1681246.13 [0;2822833],0.59 [0;0],19.47 [0;35],0.6 [0;0],0.58 [0;0],0.4 [0;1],0.8 [0;2],0.6 [0;1],0.6 [0;1],0.6 [0;0]
MetaHipMer2,Multiple k-mer De Bruijn graph,14.73 [0;27],0.07 [0;0],65553.45 [1166;118895],63736.91 [0;118895],0.0 [0;0],Metagenomic,1583777.36 [9974;2811369],0.56 [0;0],73.82 [5;179],1.0 [0;1],1.0 [0;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1]
metaSPAdes,Multiple k-mer De Bruijn graph,9.0 [0;25],0.1 [0;0],124052.6 [0;238611],124052.6 [0;238611],0.0 [0;0],Metagenomic,1683765.2 [0;2818677],0.59 [0;0],19.0 [0;51],0.6 [0;0],0.59 [0;0],0.2 [0;1],0.4 [0;2],0.6 [0;1],0.6 [0;1],0.6 [0;1]
MINIA,Single k-mer De Bruijn graph,71.07 [0;149],0.02 [0;0],18014.0 [0;30453],16138.2 [0;30453],0.0 [0;0],Genomic,1658143.47 [0;2770364],0.58 [0;0],111.6 [0;222],0.87 [0;1],0.86 [0;1],0.0 [0;0],0.0 [0;0],0.87 [0;1],0.87 [0;1],0.87 [0;1]
SKESA,Multiple k-mer De Bruijn graph,18.2 [0;42],0.07 [0;0],64106.4 [0;152709],63471.4 [0;152709],0.0 [0;0],Genomic,1683198.4 [0;2810284],0.59 [0;0],33.87 [0;72],0.86 [0;1],0.7 [0;1],0.4 [0;1],0.4 [0;1],0.87 [0;1],0.87 [0;1],0.86 [0;1]


Escherichia_coli_plasmid


...

,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,0.0 [0;0],0.5 [0;0],3167.25 [0;6335],3167.25 [0;6335],0.0 [0;0],Metagenomic,3167.25 [0;6335],0.5 [0;0],0.5 [0;1],0.5 [0;1],0.5 [0;1],0.0 [0;0],0.0 [0;0],0.5 [0;1],0.5 [0;1],0.5 [0;1]
BCALM2,Single k-mer De Bruijn graph,0.0 [0;0],0.2 [0;0],1267.2 [0;6336],1267.2 [0;6336],0.0 [0;0],Genomic,1267.2 [0;6336],0.2 [0;0],0.2 [0;1],0.2 [0;1],0.2 [0;1],0.0 [0;0],0.0 [0;0],0.2 [0;1],0.2 [0;1],0.2 [0;1]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0 [0;0],0.6 [0;1],3837.27 [0;6522],3837.27 [0;6522],0.0 [0;0],Metagenomic,3801.4 [0;6337],0.6 [0;1],0.6 [0;1],0.6 [0;1],0.6 [0;1],0.0 [0;0],0.0 [0;0],0.61 [0;1],0.61 [0;1],0.6 [0;1]
IDBA-UD,Multiple k-mer De Bruijn graph,0.0 [0;0],0.55 [0;0],3487.4 [0;6336],3487.4 [0;6336],0.0 [0;0],Metagenomic,3481.4 [0;6336],0.55 [0;0],0.6 [0;1],0.6 [0;1],0.6 [0;1],0.0 [0;0],0.0 [0;0],0.6 [0;1],0.6 [0;1],0.6 [0;1]
MEGAHIT,Multiple k-mer De Bruijn graph,0.0 [0;0],0.6 [0;1],3887.8 [0;6795],3887.8 [0;6795],0.0 [0;0],Metagenomic,3796.2 [0;6337],0.6 [0;1],0.6 [0;1],0.59 [0;1],0.59 [0;1],0.0 [0;0],0.0 [0;0],0.6 [0;1],0.61 [0;1],0.59 [0;1]
MetaHipMer2,Multiple k-mer De Bruijn graph,0.0 [0;0],0.55 [0;0],3454.91 [0;6334],3454.91 [0;6334],0.0 [0;0],Metagenomic,3454.91 [0;6334],0.55 [0;0],0.55 [0;1],0.55 [0;1],0.55 [0;1],0.0 [0;0],0.0 [0;0],0.55 [0;1],0.55 [0;1],0.55 [0;1]
metaSPAdes,Multiple k-mer De Bruijn graph,0.2 [0;1],0.53 [0;0],3381.8 [0;6336],3381.8 [0;6336],0.0 [0;0],Metagenomic,3775.2 [0;6336],0.6 [0;0],0.8 [0;2],0.6 [0;1],0.6 [0;1],0.0 [0;0],0.0 [0;0],0.6 [0;1],0.6 [0;1],0.6 [0;1]
MINIA,Single k-mer De Bruijn graph,0.0 [0;0],0.4 [0;0],2528.6 [0;6336],2528.6 [0;6336],0.0 [0;0],Genomic,2528.6 [0;6336],0.4 [0;0],0.4 [0;1],0.4 [0;1],0.4 [0;1],0.0 [0;0],0.0 [0;0],0.4 [0;1],0.4 [0;1],0.4 [0;1]
SKESA,Multiple k-mer De Bruijn graph,0.0 [0;0],0.43 [0;0],2736.6 [0;6044],2736.6 [0;6044],0.0 [0;0],Genomic,3374.8 [0;6044],0.53 [0;0],1.0 [0;3],0.6 [0;1],0.59 [0;1],0.0 [0;0],0.0 [0;0],0.6 [0;1],0.6 [0;1],0.6 [0;1]


Staphylococcus_aureus_plasmid2


,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,0.0 [0;0],0.6 [0;0],1336.67 [0;2215],1336.67 [0;2215],0.0 [0;0],Metagenomic,1336.67 [0;2215],0.6 [0;0],0.67 [0;1],0.67 [0;1],0.67 [0;1],0.0 [0;0],0.0 [0;0],0.67 [0;1],0.67 [0;1],0.67 [0;1]
BCALM2,Single k-mer De Bruijn graph,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],Genomic,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0 [0;0],0.61 [0;1],1355.2 [0;2397],1355.2 [0;2397],0.0 [0;0],Metagenomic,1321.4 [0;2216],0.6 [0;1],0.6 [0;1],0.6 [0;1],0.6 [0;1],0.0 [0;0],0.0 [0;0],0.62 [0;1],0.62 [0;1],0.6 [0;1]
IDBA-UD,Multiple k-mer De Bruijn graph,0.0 [0;0],0.41 [0;1],906.0 [0;2315],906.0 [0;2315],0.0 [0;0],Metagenomic,886.2 [0;2216],0.4 [0;1],0.4 [0;1],0.4 [0;1],0.4 [0;1],0.0 [0;0],0.0 [0;0],0.41 [0;1],0.41 [0;1],0.4 [0;1]
MEGAHIT,Multiple k-mer De Bruijn graph,0.0 [0;0],0.61 [0;1],1357.4 [0;2357],1357.4 [0;2357],0.0 [0;0],Metagenomic,1329.2 [0;2216],0.6 [0;1],0.6 [0;1],0.6 [0;1],0.6 [0;1],0.0 [0;0],0.0 [0;0],0.61 [0;1],0.61 [0;1],0.6 [0;1]
MetaHipMer2,Multiple k-mer De Bruijn graph,0.0 [0;0],0.54 [0;0],1207.09 [0;2213],1207.09 [0;2213],0.0 [0;0],Metagenomic,1207.09 [0;2213],0.54 [0;0],0.55 [0;1],0.55 [0;1],0.55 [0;1],0.0 [0;0],0.0 [0;0],0.55 [0;1],0.55 [0;1],0.55 [0;1]
metaSPAdes,Multiple k-mer De Bruijn graph,0.0 [0;0],0.58 [0;1],1284.6 [0;2271],1284.6 [0;2271],0.0 [0;0],Metagenomic,1273.6 [0;2216],0.57 [0;1],0.6 [0;1],0.6 [0;0],0.6 [0;0],0.0 [0;0],0.0 [0;0],0.6 [0;1],0.6 [0;1],0.6 [0;1]
MINIA,Single k-mer De Bruijn graph,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],Genomic,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0]
SKESA,Multiple k-mer De Bruijn graph,0.0 [0;0],0.48 [0;0],1074.4 [0;2005],1074.4 [0;2005],0.0 [0;0],Genomic,1074.4 [0;2005],0.48 [0;0],0.6 [0;1],0.6 [0;1],0.6 [0;1],0.0 [0;0],0.0 [0;0],0.6 [0;1],0.6 [0;1],0.6 [0;1]


Staphylococcus_aureus_plasmid3


,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,0.0 [0;0],0.61 [0;0],1821.58 [0;2992],1821.58 [0;2992],0.0 [0;0],Metagenomic,1821.58 [0;2992],0.61 [0;0],0.67 [0;1],0.67 [0;1],0.67 [0;1],0.0 [0;0],0.0 [0;0],0.67 [0;1],0.67 [0;1],0.67 [0;1]
BCALM2,Single k-mer De Bruijn graph,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],Genomic,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0 [0;0],0.6 [0;1],1785.8 [0;3162],1785.8 [0;3162],0.0 [0;0],Metagenomic,1754.8 [0;2993],0.59 [0;1],0.6 [0;1],0.6 [0;1],0.6 [0;1],0.0 [0;0],0.0 [0;0],0.61 [0;1],0.61 [0;1],0.6 [0;1]
IDBA-UD,Multiple k-mer De Bruijn graph,0.2 [0;1],0.49 [0;0],1476.4 [0;2847],1476.4 [0;2847],0.0 [0;0],Metagenomic,1690.2 [0;2847],0.56 [0;0],0.8 [0;2],0.6 [0;1],0.6 [0;1],0.0 [0;0],0.0 [0;0],0.61 [0;1],0.61 [0;1],0.6 [0;1]
MEGAHIT,Multiple k-mer De Bruijn graph,0.0 [0;0],0.61 [0;1],1823.6 [0;3134],1823.6 [0;3134],0.0 [0;0],Metagenomic,1795.4 [0;2993],0.6 [0;1],0.6 [0;1],0.6 [0;1],0.6 [0;1],0.0 [0;0],0.0 [0;0],0.61 [0;1],0.61 [0;1],0.6 [0;1]
MetaHipMer2,Multiple k-mer De Bruijn graph,0.0 [0;0],0.69 [0;0],2065.36 [0;2990],2065.36 [0;2990],0.0 [0;0],Metagenomic,2065.36 [0;2990],0.69 [0;0],0.82 [0;1],0.82 [0;1],0.82 [0;1],0.0 [0;0],0.0 [0;0],0.82 [0;1],0.82 [0;1],0.82 [0;1]
metaSPAdes,Multiple k-mer De Bruijn graph,0.4 [0;1],0.48 [0;1],1442.2 [0;3857],1442.2 [0;3857],0.0 [0;0],Metagenomic,1766.4 [0;2993],0.59 [0;1],1.0 [0;2],0.6 [0;1],0.6 [0;1],0.0 [0;0],0.0 [0;0],0.67 [0;1],0.67 [0;1],0.6 [0;1]
MINIA,Single k-mer De Bruijn graph,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],Genomic,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0]
SKESA,Multiple k-mer De Bruijn graph,0.0 [0;0],0.51 [0;0],1522.0 [0;2782],1522.0 [0;2782],0.0 [0;0],Genomic,1522.0 [0;2782],0.51 [0;0],0.6 [0;1],0.6 [0;1],0.6 [0;1],0.0 [0;0],0.0 [0;0],0.6 [0;1],0.6 [0;1],0.6 [0;1]


#### Per reference replicon for all assemblers - Even only!

In [25]:
ref_stats_per_reference_all_assemblers = pd.DataFrame()
for reference in reference_pipeline_metrics_df.reference.unique():
    row={'Reference': reference}
    lala=reference_pipeline_metrics_df[(reference_pipeline_metrics_df.reference == reference) & (reference_pipeline_metrics_df.distribution == 'Even')].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    ref_stats_per_reference_all_assemblers=ref_stats_per_reference_all_assemblers.append(row, ignore_index=True)
ref_stats_per_reference_all_assemblers = ref_stats_per_reference_all_assemblers.set_index('Reference')
ref_stats_per_reference_all_assemblers.to_csv("Tables/Results/Reference metrics per replicon.csv")
display(ref_stats_per_reference_all_assemblers)

,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Reference,,,,,,,,,,,,,,,
Bacillus_subtilis,25.64 [0;120],0.1 [0;0],196509.03 [0;343843],194589.61 [0;343843],1.32 [0;47],3614434.53 [0;3991152],0.89 [0;0],91.85 [0;1467],0.94 [0;1],0.92 [0;1],0.08 [0;2],0.2 [0;4],0.94 [0;1],0.94 [0;1],0.94 [0;1]
Enterococcus_faecalis,36.26 [0;149],0.13 [0;0],134153.57 [0;377104],132615.19 [0;377104],0.0 [0;0],2491484.12 [0;2828460],0.88 [0;0],72.49 [0;231],0.94 [0;1],0.91 [0;1],0.34 [0;3],0.62 [0;6],0.95 [0;1],0.95 [0;1],0.94 [0;1]
Escherichia_coli_plasmid,3.27 [0;18],0.22 [0;0],18003.78 [0;55639],17461.25 [0;55639],37.6 [0;835],82423.2 [0;109516],0.75 [0;0],9.2 [0;33],0.89 [0;1],0.88 [0;1],0.0 [0;0],0.0 [0;0],0.89 [0;1],0.89 [0;1],0.89 [0;1]
Escherichia_coli,70.08 [0;283],0.04 [0;0],51151.78 [0;138793],50851.09 [0;138793],1.26 [0;45],4051935.78 [0;4706885],0.85 [0;0],363.8 [0;1495],0.94 [0;1],0.84 [0;1],1.07 [0;8],1.99 [0;15],0.95 [0;1],0.95 [0;1],0.94 [0;1]
Lactobacillus_fermentum,77.7 [0;263],0.05 [0;0],30397.63 [0;46932],29158.61 [0;52972],0.0 [0;0],1585069.36 [0;1873053],0.83 [0;0],105.8 [0;282],0.92 [0;1],0.9 [0;1],0.34 [0;4],0.69 [0;8],0.92 [0;1],0.92 [0;1],0.92 [0;1]
Listeria_monocytogenes,24.36 [0;122],0.15 [0;0],236971.75 [0;542872],228275.5 [0;542872],0.03 [0;1],2625730.37 [0;2961845],0.88 [0;0],62.41 [0;361],0.94 [0;1],0.93 [0;1],0.03 [0;1],0.06 [0;2],0.95 [0;1],0.95 [0;1],0.94 [0;1]
Pseudomonas_aeruginosa,147.13 [0;1004],0.06 [0;0],141661.78 [0;338923],146649.92 [0;464977],583.01 [0;12098],6111755.96 [0;6780808],0.9 [0;0],240.31 [0;1404],0.94 [0;1],0.85 [0;1],1.27 [0;8],2.26 [0;15],0.95 [0;1],0.95 [0;1],0.94 [0;1]
Salmonella_enterica,54.45 [0;200],0.05 [0;0],71283.69 [0;223960],70301.45 [0;223960],0.12 [0;2],4074863.96 [0;4703394],0.86 [0;0],332.36 [0;1479],0.94 [0;1],0.83 [0;1],0.79 [0;9],1.25 [0;14],0.95 [0;1],0.95 [0;1],0.94 [0;1]
Staphylococcus_aureus,40.08 [0;140],0.12 [0;0],114085.93 [0;394879],113359.04 [0;394879],1.99 [0;71],2373450.96 [0;2693907],0.87 [0;0],76.07 [0;247],0.94 [0;1],0.91 [0;1],0.27 [0;3],0.63 [0;7],0.94 [0;1],0.95 [0;1],0.94 [0;1]


### Reference stats per distribution

In [26]:
for distribution in reference_pipeline_metrics_df.distribution.unique():
    ref_stats_per_reference_per_assembler = pd.DataFrame()
    for assembler in reference_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
             'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=reference_pipeline_metrics_df[(reference_pipeline_metrics_df.assembler == assembler) & (reference_pipeline_metrics_df.distribution == distribution)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            minimum = lala.loc['min',column]
            maximum = lala.loc['max',column]
            row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
        ref_stats_per_reference_per_assembler=ref_stats_per_reference_per_assembler.append(row, ignore_index=True)
    ref_stats_per_reference_per_assembler = ref_stats_per_reference_per_assembler.set_index('Assembler')
    ref_stats_per_reference_per_assembler.to_csv("Tables/Results/{} - Reference metrics per assembler.csv".format(distribution))
    print(distribution)
    display(ref_stats_per_reference_per_assembler)

Log


,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,50.54 [0;1033],0.02 [0;0],25986.46 [0;351511],25866.25 [0;351511],16.67 [0;400],Metagenomic,690335.21 [0;6781605],0.16 [0;0],65.12 [0;1141],0.21 [0;0],0.2 [0;0],0.0 [0;0],0.0 [0;0],0.21 [0;1],0.21 [0;1],0.21 [0;1]
BCALM2,Single k-mer De Bruijn graph,13.58 [0;311],0.0 [0;0],3455.14 [0;64777],3263.39 [0;62493],0.0 [0;0],Genomic,390195.64 [0;6515102],0.09 [0;0],35.86 [0;496],0.17 [0;1],0.17 [0;1],0.0 [0;0],0.0 [0;0],0.17 [0;1],0.17 [0;1],0.17 [0;1]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,23.6 [0;459],0.02 [0;0],41255.35 [0;351149],40885.01 [0;351149],0.0 [0;0],Metagenomic,1131019.22 [0;6716046],0.25 [0;0],37.54 [0;604],0.36 [0;1],0.36 [0;1],0.08 [0;2],0.25 [0;6],0.36 [0;1],0.36 [0;1],0.36 [0;1]
IDBA-UD,Multiple k-mer De Bruijn graph,16.33 [0;292],0.02 [0;0],48013.69 [0;475377],46431.78 [0;475377],0.0 [0;0],Metagenomic,1135250.6 [0;6700232],0.25 [0;0],29.56 [0;418],0.46 [0;0],0.38 [0;0],0.25 [0;2],0.39 [0;5],0.47 [0;1],0.61 [0;5],0.44 [0;1]
MEGAHIT,Multiple k-mer De Bruijn graph,8.42 [0;147],0.03 [0;0],66650.44 [0;543128],66455.53 [0;543128],0.0 [0;0],Metagenomic,1141247.44 [0;6738936],0.25 [0;0],18.11 [0;223],0.46 [0;0],0.42 [0;0],0.06 [0;1],0.11 [0;2],0.47 [0;1],0.48 [0;1],0.47 [0;1]
MetaHipMer2,Multiple k-mer De Bruijn graph,0.0 [0;0],0.0 [0;0],903.29 [0;2986],529.12 [0;2697],0.0 [0;0],Metagenomic,825097.88 [0;3766506],0.19 [0;0],407.83 [0;2404],0.42 [0;1],0.39 [0;1],0.0 [0;0],0.0 [0;0],0.42 [0;1],0.42 [0;1],0.42 [0;1]
metaSPAdes,Multiple k-mer De Bruijn graph,7.19 [0;66],0.03 [0;0],60717.78 [0;593167],59453.89 [0;593167],0.0 [0;0],Metagenomic,1140707.44 [0;6733500],0.25 [0;0],15.53 [0;110],0.47 [0;0],0.43 [0;0],0.17 [0;2],0.39 [0;6],0.47 [0;1],0.48 [0;1],0.47 [0;1]
MINIA,Single k-mer De Bruijn graph,65.1 [0;649],0.01 [0;0],6383.53 [0;64777],5783.75 [0;64473],0.0 [0;0],Genomic,1099789.64 [0;6524118],0.24 [0;0],119.54 [0;919],0.42 [0;1],0.39 [0;1],0.14 [0;2],0.22 [0;4],0.42 [0;1],0.42 [0;1],0.42 [0;1]
SKESA,Multiple k-mer De Bruijn graph,6.72 [0;232],0.01 [0;0],10421.97 [0;351027],10210.64 [0;351027],0.0 [0;0],Genomic,366940.31 [0;3512871],0.1 [0;0],120.94 [0;1877],0.19 [0;1],0.19 [0;1],0.33 [0;12],0.64 [0;23],0.19 [0;1],0.19 [0;1],0.19 [0;1]


Even


,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,60.4 [0;1004],0.3 [0;0],101808.53 [0;351502],101667.72 [0;351502],92.14 [0;2170],Metagenomic,2534947.7 [0;6780808],0.91 [0;0],97.11 [0;1404],0.98 [0;1],0.94 [0;1],0.33 [0;8],0.59 [0;14],0.99 [0;1],0.99 [0;1],0.98 [0;1]
BCALM2,Single k-mer De Bruijn graph,51.2 [0;921],0.04 [0;0],6932.33 [0;65138],6539.94 [0;64934],0.0 [0;0],Genomic,1236062.67 [0;6510588],0.37 [0;0],272.97 [0;1495],0.5 [0;1],0.5 [0;1],0.0 [0;0],0.0 [0;0],0.5 [0;1],0.5 [0;1],0.5 [0;1]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,31.33 [0;112],0.35 [0;1],106093.76 [2176;452364],96782.73 [2176;351186],0.0 [0;0],Metagenomic,2523829.08 [2176;6716522],0.98 [0;1],59.92 [1;199],1.0 [0;1],0.98 [0;1],0.14 [0;3],0.25 [0;6],1.01 [0;1],1.01 [0;1],1.0 [0;1]
IDBA-UD,Multiple k-mer De Bruijn graph,68.82 [0;283],0.27 [0;1],55836.03 [0;254629],54554.11 [0;254629],0.0 [0;0],Metagenomic,2507600.54 [0;6710005],0.94 [0;1],121.12 [0;449],0.97 [0;1],0.87 [0;1],1.12 [0;9],1.94 [0;15],0.98 [0;1],0.98 [0;1],0.97 [0;1]
MEGAHIT,Multiple k-mer De Bruijn graph,18.31 [0;55],0.37 [0;1],138989.33 [2215;542872],141793.83 [2215;542872],0.0 [0;0],Metagenomic,2534425.59 [2215;6737986],0.99 [0;1],44.39 [1;141],1.0 [0;1],0.93 [0;1],0.97 [0;4],1.99 [0;7],1.0 [0;1],1.01 [0;1],1.0 [0;1]
MetaHipMer2,Multiple k-mer De Bruijn graph,21.7 [0;110],0.23 [0;0],58464.9 [0;257424],57747.71 [0;257424],0.0 [0;0],Metagenomic,1722404.01 [0;6717767],0.68 [0;0],74.19 [0;361],0.94 [0;1],0.94 [0;1],0.0 [0;0],0.0 [0;0],0.94 [0;1],0.94 [0;1],0.94 [0;1]
metaSPAdes,Multiple k-mer De Bruijn graph,20.14 [0;90],0.34 [0;1],133559.97 [1677;475510],132927.86 [1677;475510],0.0 [0;0],Metagenomic,2530047.11 [2076;6726884],0.98 [0;1],39.33 [1;167],1.0 [0;1],0.99 [0;1],0.14 [0;2],0.33 [0;6],1.01 [0;1],1.01 [0;1],1.0 [0;1]
MINIA,Single k-mer De Bruijn graph,83.41 [0;407],0.08 [0;0],17271.52 [0;65138],16622.5 [0;64960],0.0 [0;0],Genomic,2345242.24 [0;6518607],0.74 [0;0],307.84 [0;1147],0.81 [0;1],0.79 [0;1],0.0 [0;0],0.0 [0;0],0.81 [0;1],0.81 [0;1],0.81 [0;1]
SKESA,Multiple k-mer De Bruijn graph,57.92 [0;670],0.27 [0;0],76053.75 [1404;351014],75134.92 [0;351014],0.0 [0;0],Genomic,2496709.31 [1460;6673601],0.9 [0;0],96.06 [1;933],1.0 [0;1],0.87 [0;1],0.67 [0;7],1.17 [0;15],1.0 [0;1],1.0 [0;1],1.0 [0;1]


### Genomes vs Plasmids

In [27]:
ref_stats_per_replicon_type = pd.DataFrame()
# Genome
row={'Replicon': "Genome"}
lala=reference_pipeline_metrics_df[~reference_pipeline_metrics_df['reference'].str.contains("plasmid")].describe()
for column in lala.columns:
    mean = lala.loc['mean',column]
    std = lala.loc['std',column]
    row[column] = "{}(± {})".format(round(mean, 2), round(std, 2))
ref_stats_per_replicon_type =ref_stats_per_replicon_type .append(row, ignore_index=True)

# Plasmid
row={'Replicon': "Plasmid"}
lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df['reference'].str.contains("plasmid")].describe()
for column in lala.columns:
    mean = lala.loc['mean',column]
    minimum = lala.loc['min',column]
    maximum = lala.loc['max',column]
    row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    
ref_stats_per_replicon_type =ref_stats_per_replicon_type .append(row, ignore_index=True)
ref_stats_per_replicon_type  = ref_stats_per_replicon_type .set_index('Replicon')
ref_stats_per_replicon_type.to_csv("Tables/Results/Reference metrics per replicon type.csv")
display(ref_stats_per_replicon_type)

,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Replicon,,,,,,,,,,,,,,,
Genome,47.95(± 107.54),0.06(± 0.08),93543.9(± 121264.69),92430.74(± 121287.75),57.03(± 683.46),2585647.14(± 2155710.78),0.65(± 0.45),147.71(± 307.44),0.78(± 0.41),0.73(± 0.41),1.23(± 21.49),2.4(± 43.13),0.78(± 0.41),0.79(± 0.46),0.78(± 0.41)
Plasmid,0.52 [0;18],0.36 [0;1],4081.6 [0;55639],3985.8 [0;55639],5.81 [0;835],14065.64 [0;109516],0.44 [0;1],1.8 [0;33],0.49 [0;1],0.49 [0;1],0.0 [0;0],0.0 [0;0],0.49 [0;1],0.5 [0;1],0.49 [0;1]
